In [42]:
import requests
import json, re, time
from datetime import datetime
import pandas as pd

In [43]:
with open('api.key','r') as f:
    APIKEY = f.read()

HEADERS = {'apikey': APIKEY,
            'accept': 'application/json',
            'content-type': 'application/json',
            'cookie': 'ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip; ASP.NET_SessionId=aiggen1ccck0gq141dgq1sip'
          }

API_ENDPOINT = 'https://kong.speedcheckerapi.com:8443/ProbeAPIv2/'

probeInfoProperties = [
        "ASN",
        "CityName",
        "ConnectionType",
        "CountryCode",
        "DNSResolver",
        "GeolocationAccuracy",
        "IPAddress",
        "Latitude",
        "Longitude",
        "Network",
        "NetworkID",
        "Platform",
        "ProbeID",
        "Version"
    ]

In [44]:
def getTestSettings(cc, url):
    
    destinations = [url]
    json_test = {
                  "testSettings": {
                    "TestCount": 10,
                    "Sources": [
                      {
                        "CountryCode": cc
                      }
                    ],
                    "Destinations": destinations,
                    "ProbeInfoProperties": probeInfoProperties
                  }
                }
    return json_test

### StartPageLoadTest function

In [45]:
def startPageLoadTest(test_settings):
    
    test_url = API_ENDPOINT + "StartPageLoadTest"
    try:
        r = requests.post(test_url, data=json.dumps(test_settings), headers=HEADERS)
    except requests.exceptions.RequestException as e:
        return "Request FAILED"
    
    res = json.loads(r.text)
        
    if ("OK" == res['StartPageLoadTestResult']['Status']['StatusText']):
        return res['StartPageLoadTestResult']['TestID']
    else:
        return "FAILED"

### retrievePageLoadTestResults function

In [46]:
def retrievePageLoadTestResults(testID):
    
    url = API_ENDPOINT + "GetPageLoadResults?apikey=" + APIKEY + "&testID=" + testID
    
    try:    
        r = requests.get(url, headers=HEADERS)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return []
    
    res = json.loads(r.text)
    return res['PageLoadTestResults']

## Retrieve results

In [47]:
def getResultsFromTestIDs(df_tests_url):
    df_results = pd.DataFrame(columns=['domain',
                                       'cc',
                                       'ProbeInfo',
                                       'TestDateTime',
                                       'StatusCode',
                                       'StatusText',
                                       'DNSLookupTime', 
                                       'Destination', 
                                       'HTTPStatus', 
                                       'InitialConnection', 
                                       'NumberOfRequests', 
                                       'PageLoadTime', 
                                       'SSLNegotiationTime', 
                                       'TTFB', 
                                       'TotalDownloadedBytes'
                                      ])
    
    #Exclude FAILED items
    for index, row in df_tests_url.loc[df_tests_url.testID!='FAILED'].iterrows():
        
        domain = row['domain']
        cc = row['cc']
        testID = row['testID']
        pageLoadTestResults =  retrievePageLoadTestResults(testID)

        if (pageLoadTestResults is None or len(pageLoadTestResults) < 1):
            continue
        
        for res in pageLoadTestResults:
        
            probeInfo = res['ProbeInfo']
            testDateTime = res['TestDateTime']

            #extract the epoch
            m = re.search('/Date\((\d+)\+0000\)/', testDateTime)
            if m:
                testDateTime = m.group(1)[:-3]

            #convert into human readable format
            testDateTime = datetime.fromtimestamp(int(testDateTime)).strftime('%Y-%m-%d')
            statusCode = res['TestStatus']['StatusCode']
            statusText = res['TestStatus']['StatusText']
            dnsLookupTime = res['DNSLookupTime']
            destination =  res['Destination']
            hTTPStatus = res['HTTPStatus']
            initialConnection = res['InitialConnection']
            numberOfRequests =  res['NumberOfRequests'] 
            pageLoadTime =  res['PageLoadTime'] 
            sslNegotiationTime = res['SSLNegotiationTime'] 
            ttfb = res['TTFB']
            totalDownloadedBytes = res['TotalDownloadedBytes']

            df_results = df_results.append({
                'domain': domain,
                'cc': cc,
                'ProbeInfo':probeInfo,
                'TestDateTime': testDateTime,
                'StatusCode':statusCode,
                'StatusText':statusText,
                'DNSLookupTime':dnsLookupTime,
                'Destination':destination,
                'HTTPStatus':hTTPStatus,
                'InitialConnection':initialConnection,
                'NumberOfRequests':numberOfRequests,
                'PageLoadTime':pageLoadTime,
                'SSLNegotiationTime':sslNegotiationTime,
                'TTFB':ttfb,
                'TotalDownloadedBytes':totalDownloadedBytes
            }, ignore_index=True)

    return df_results

In [48]:
def runMeasurements(df_links, url_type, filename):
    df_testID = pd.DataFrame(columns=['domain', 'cc', 'testID'])

    for index, row in df_links.iterrows():
        domain = row['domain']
        cc = row['cc']
        url = row[url_type]

        test_settings = getTestSettings(cc, url)
        testID = startPageLoadTest(test_settings)
        print(testID)
        
        df_testID = df_testID.append({'domain': domain, 'cc': cc, 'testID': testID}, ignore_index=True)
    
    df_testID.to_csv(filename, sep='|', encoding='utf-8', index=False)    
    

In [49]:
df_links = pd.read_csv('data/links_sample.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [50]:
runMeasurements(df_links, 'url', 'data/url_testID.csv')
#runMeasurements(df_links, 'ori_amp_url', 'data/ori_amp_url_testID_4.csv')
#runMeasurements(df_links, 'amp_viewer_url', 'data/amp_viewer_url_testID2.csv')
#runMeasurements(df_links, 'amp_cdn_url', 'data/amp_cdn_url_testID.csv')

dc3b499c-4dc9-45cf-ae68-7cc902179a4c
46f4aee1-bf14-487c-aaca-a4d1436a7f70
cbffd553-e24a-48fb-b7c0-9b11dd0fe88a
d0770d98-942a-44af-bf2f-b9da8e3b61b3
FAILED
d8d591ea-4349-4c18-9001-ff452bc4f99b
577ff56e-8f1e-48ea-84c3-0e710223556b
97d3937f-aa6e-4577-bb23-49e07a948ad6
a61b9882-bc92-45fb-943d-e691f02fe578
d547b549-fb88-4905-806e-f44926ac367e
9a7f809d-149c-4165-9a71-bbf713435bfe
adb2b614-ce5b-4df3-bdaa-4b29307360a5
c952df8f-91b1-402a-96be-f83f3e455538
b0c07ec9-79dc-4df1-a0d7-3934487ede43
a8bd93eb-b26e-4677-a273-50ac7ab21e5e
17104fcd-9ef8-404d-808b-ec8c55bed742
e49d9e74-88cc-40f6-947d-faa96435b81b
67310bc4-c5c3-44fc-834e-9c5083dbf38e
f249e537-04e6-4f73-a36f-50b04ef3a3e1
8c32c474-a73f-4db5-8e92-a7b96b5cbdb1
cd8401bb-2758-40f5-95fd-a4423119a235
a120831d-35ac-4576-acb8-841ac1a7b74d
44a34ea9-4dbf-4dd9-8957-e18230afb42c
66d8d953-a4cb-4ecd-9062-f1f1ce4a74c9
c0d112b5-fcd8-41a5-a8a7-6b3329167803
e126d84d-e389-46ca-8942-8c474d1359d9
0a622258-2ed6-45fc-ba98-8a878f9c6b5f
71b9abcd-8e91-4861-8ca5-202cfed

In [51]:
df_testIDs_url = pd.read_csv('data/url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_ori_amp = pd.read_csv('data/ori_amp_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_viewer = pd.read_csv('data/amp_viewer_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)
#df_testIDs_amp_cdn = pd.read_csv('data/amp_cdn_url_testID.csv', sep='|', encoding='utf-8', keep_default_na=False)

In [52]:
#df_results_ori_amp = getResultsFromTestIDs(df_testIDs_ori_amp)
#df_results_amp_viewer = getResultsFromTestIDs(df_testIDs_amp_viewer)
#df_results_amp_cdn = getResultsFromTestIDs(df_testIDs_amp_cdn)
df_results_url = getResultsFromTestIDs(df_testIDs_url)

In [53]:
df_results_url.to_csv("data/url_results" + time.strftime("%d%m%Y") + ".csv", sep='|', encoding='utf-8', index=False)
#df_results_ori_amp.to_csv("data/ori_amp_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_viewer.to_csv("data/amp_viewer_url_results.csv", sep='|', encoding='utf-8', index=False)
#df_results_amp_cdn.to_csv("data/amp_cdn_url_results.csv", sep='|', encoding='utf-8', index=False)

In [55]:
df_results_url = df_results_url[(df_results_url.HTTPStatus=='200') & (df_results_url.StatusText=='OK')]

In [57]:
df_results_url.to_csv("data/url_results.csv", sep='|', encoding='utf-8', index=False)
